<a href="https://colab.research.google.com/github/hhkbhamza/BrainTumorClassification/blob/main/webappforbraintumorclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install streamlit pyngrok python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [2]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata
import os

In [3]:
ngrok_token = userdata.get('NGROK_AUTH_TOKEN')

ngrok.set_auth_token(ngrok_token)
print(ngrok_token)

2ozdfbYYnWm9f6T2GgK4QpjBZeu_BCMMFMXAQUAnHNhzzGiA


In [4]:
def run_streamlit():
  os.system("streamlit run /content/app.py --server.port 8500")

In [28]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import plotly.graph_objects as go
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
import google.generativeai as genai
from google.colab import userdata
import PIL.Image
import os
from dotenv import load_dotenv
load_dotenv()

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

output_dir = 'saliency_maps'
os.makedirs(output_dir, exist_ok=True)

def generate_explanation(img_path, model_prediction, confidence):
    prompt = f"""You are an expert neurologist. You are tasked with explaining a saliency map of a brain tumor MRI scan.
    The saliency map was generated by a deep learning model that was trained to classify brain tumors
    as either glioma, meningioma, pituitary, or no tumor.

    The saliency map highlights the regions of the image that the machine learning model is focusing on to make the prediction.

    The deep learning model predicted the image to be of class '{model_prediction}' with a confidence of {confidence * 100}%.

    In your response:
    - Explain what regions of the brain the model is focusing on, based on the saliency map. Refer to the regions highlighted
      in light cyan, those are the regions where the model is focusing on.
    - Explain possible reasons why the model made the prediction it did.
    - Don't mention anything like 'The saliency map highlights the regions the model is focusing on, which are in light cyan'
      in your explanation.
    - Keep your explanation to 4 sentences max.

    Let's think step by step about this. Verify step by step.
    """

    img = PIL.Image.open(img_path)

    model = genai.GenerativeModel(model_name="gemini-1.5-flash")
    response = model.generate_content([prompt, img])

    return response.text



def generate_saliency_map(model, img_array, class_index, img_size):
    with tf.GradientTape() as tape:
        img_tensor = tf.convert_to_tensor(img_array)
        tape.watch(img_tensor)
        predictions = model(img_tensor)
        target_class = predictions[:, class_index]

    gradients = tape.gradient(target_class, img_tensor)
    gradients = tf.math.abs(gradients)
    gradients = tf.reduce_max(gradients, axis=-1)
    gradients = gradients.numpy().squeeze()

    # Resize gradients to match original image size
    gradients = cv2.resize(gradients, img_size)

    # Create a circular mask for the brain area
    center = (gradients.shape[0] // 2, gradients.shape[1] // 2)
    radius = min(center[0], center[1]) - 10
    x, y = np.ogrid[:gradients.shape[0], :gradients.shape[1]]
    mask = (x - center[0]) ** 2 + (y - center[1]) ** 2 <= radius ** 2

    # Apply mask to gradients
    gradients = gradients * mask

    # Normalize only the brain area
    brain_gradients = gradients[mask]
    if brain_gradients.max() > brain_gradients.min():
        brain_gradients = (brain_gradients - brain_gradients.min()) / (brain_gradients.max() - brain_gradients.min())
    gradients[mask] = brain_gradients

    # Apply a higher threshold
    threshold = np.percentile(gradients[mask], 80)
    gradients[gradients < threshold] = 0

    # Apply more aggressive smoothing
    gradients = cv2.GaussianBlur(gradients, (11, 11), 0)

    # Create a heatmap overlay with enhanced contrast
    heatmap = cv2.applyColorMap(np.uint8(255 * gradients), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Resize heatmap to match original image size
    heatmap = cv2.resize(heatmap, img_size)

    # Superimpose the heatmap on the original image with increased opacity
    original_img = image.img_to_array(img)
    superimposed_img = heatmap * 0.7 + original_img * 0.3
    superimposed_img = superimposed_img.astype(np.uint8)

    img_path = os.path.join(output_dir, uploaded_file.name)
    with open(img_path, "wb") as f:
      f.write(uploaded_file.getbuffer())

    saliency_map_path = f'saliency_maps/{uploaded_file.name}'

    # Save the saliency map
    cv2.imwrite(saliency_map_path, cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))

    return superimposed_img



def load_xception_model(model_path):
    img_shape = (299, 299, 3)
    base_model = tf.keras.applications.Xception(include_top=False, weights="imagenet",
                                                input_shape=img_shape, pooling="max")

    model = Sequential([
        base_model,
        Flatten(),
        Dropout(rate=0.3),
        Dense(128, activation="relu"),
        Dropout(rate=0.25),
        Dense(4, activation="softmax")
    ])

    model.build((None,) + img_shape)

    # Compile the model
    model.compile(Adamax(learning_rate=0.001),
                  loss="categorical_crossentropy",
                  metrics=["accuracy",
                           Precision(),
                           Recall()])

    model.load_weights(model_path)

    return model


st.title("Brain Tumor Classification")

st.write("Upload an image of a brain MRI scan to classify.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    selected_model = st.radio(
        "Select Model",
        ("Transfer Learning - Xception", "Custom CNN")
    )

    if selected_model == "Transfer Learning - Xception":
        model = load_xception_model('/content/xception_model.weights.h5')
        img_size = (299, 299)
    else:
        model = load_model('/content/cnn_model.h5')
        img_size = (224, 224)

    labels = ['Glioma', 'Meningioma', 'No tumor', 'Pituitary']
    img = image.load_img(uploaded_file, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)

    # Get the class with the highest probability
    class_index = np.argmax(prediction[0])
    result = labels[class_index]

    st.write(f"Predicted Class: {result}")
    st.write("Predictions")
    for label, prob in zip(labels, prediction[0]):
        st.write(f"{label}: {prob:.4f}")

    saliency_map = generate_saliency_map(model, img_array, class_index, img_size)

    col1, col2 = st.columns(2)
    with col1:
        st.image(uploaded_file, caption='Uploaded Image', use_container_width=True)
    with col2:
        st.image(saliency_map, caption='Saliency Map', use_container_width=True)

    st.write("## Classification Results")

    result_container = st.container()
    result_container.markdown(
        f"""
        <style>
            .result-card {{
                background: linear-gradient(145deg, #1e1e2e, #252536);
                color: #ffffff;
                padding: 25px;
                border-radius: 20px;
                box-shadow: 0 4px 6px rgba(0, 0, 0, 0.3), 0 -2px 4px rgba(0, 0, 0, 0.2);
            }}
            .result-row {{
                display: flex;
                justify-content: space-around;
                align-items: center;
            }}
            .result-column {{
                text-align: center;
                flex: 1;
                padding: 10px;
            }}
            .result-divider {{
                width: 2px;
                height: 100px;
                background-color: #444;
            }}
            .result-title {{
                margin: 0;
                font-size: 18px;
                font-weight: 600;
                color: #a8a8b5;
            }}
            .result-value {{
                margin: 5px 0 0;
                font-size: 40px;
                font-weight: 700;
            }}
            .result-prediction {{
                color: #FF6B6B;
            }}
            .result-confidence {{
                color: #4CAF50;
            }}
        </style>
        <div class="result-card">
            <div class="result-row">
                <div class="result-column">
                    <p class="result-title">Prediction</p>
                    <p class="result-value result-prediction">{result}</p>
                </div>
                <div class="result-divider"></div>
                <div class="result-column">
                    <p class="result-title">Confidence</p>
                    <p class="result-value result-confidence">{prediction[0][class_index]:.2%}</p>
                </div>
            </div>
        </div>
        </br>
        """,
        unsafe_allow_html=True
    )



    # Prepare data for Plotly chart
    probabilities = prediction[0]
    sorted_indices = np.argsort(probabilities)[::-1]
    sorted_labels = [labels[i] for i in sorted_indices]
    sorted_probabilities = probabilities[sorted_indices]

    # Create a Plotly bar chart
    fig = go.Figure()

    # Add bars to the chart with updated color scheme
    for label, prob in zip(sorted_labels, sorted_probabilities):
        color = "#FF6B6B" if label == result else "#4CAF50"
        fig.add_trace(go.Bar(
            x=[prob],
            y=[label],
            orientation='h',
            marker=dict(color=color, line=dict(color="#2D2D44", width=1)),
            text=f"{prob:.4f}",
            textposition="auto",
            hoverinfo="text"
        ))

    # Customize the chart layout
    fig.update_layout(
        title=dict(
            text="<b>Class Probabilities</b>",
            font=dict(size=20, color="#ffffff"),
            x=0.5
        ),
        xaxis=dict(
            title="<b>Probability</b>",
            tickformat=".0%",
            titlefont=dict(size=16, color="#ffffff"),
            tickfont=dict(size=14, color="#A8A8B5"),
            gridcolor="#3A3A52"
        ),
        yaxis=dict(
            title="<b>Class</b>",
            titlefont=dict(size=16, color="#ffffff"),
            tickfont=dict(size=14, color="#A8A8B5"),
            autorange="reversed",
            showgrid=False
        ),
        paper_bgcolor="#1E1E2E",
        plot_bgcolor="#252536",
        height=500,
        width=700,
        margin=dict(l=50, r=50, t=50, b=50),
        showlegend=False,
    )

    # Display the Plotly chart
    st.plotly_chart(fig, use_container_width=True)


    saliency_map_path = f'saliency_maps/{uploaded_file.name}'
    explanation = generate_explanation(saliency_map_path, result, prediction[0][class_index])

    st.write("## Explanation")
    st.write(explanation)



Overwriting app.py


In [30]:
thread = Thread(target=run_streamlit)
thread.start()

In [31]:
public_url = ngrok.connect(addr='8500', proto='http', bind_tls=True)

print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://57df-34-19-103-203.ngrok-free.app" -> "http://localhost:8500"


In [29]:
tunnels = ngrok.get_tunnels()
for tunnel in tunnels:
    print(f"Closing tunnel: {tunnel.public_url} -> {tunnel.config['addr']}")
    ngrok.disconnect(tunnel.public_url)

Closing tunnel: https://c651-34-19-103-203.ngrok-free.app -> http://localhost:8500
